# NX31 Examproject

# Introduction

## Loading packages

In [2]:
#Loads the nessecary packages
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm
from IPython.display import display

%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Enables sympy to write functions as math
sm.init_printing(use_unicode=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
examproject.example.hello_world()

'hello world'